In [2]:
import numpy as np
import json
import pickle
import copy
import whois
import re
import pandas as pd
from sklearn.metrics import confusion_matrix
import collections 
from sklearn.preprocessing import OneHotEncoder
import os
from urllib.parse import urlsplit, urlunsplit
import IP2Location
from urlextract import URLExtract
import geoip2.webservice
import urllib, sys
import torch
from sklearn.metrics import classification_report
from torch import nn
import shutil
import requests
import random
from bs4 import BeautifulSoup
from tqdm import tqdm
from copy import deepcopy
from pprint import pprint
import socket
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from nltk.corpus import words
from libs.scanner.dataset_utils import create_features_single
from libs.scanner.social_media_infogetter import InfoGetter
import numpy as np
from os import listdir
import time
from os.path import isfile, join
from progressbar import progressbar
from sklearn.decomposition import LatentDirichletAllocation
from mpl_toolkits.mplot3d import Axes3D
from transformers import pipeline
from xml.dom import minidom
import praw
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from scipy import interp
from bs4 import BeautifulSoup
from sklearn.metrics import roc_auc_score
from urllib.parse import urlparse
from captum.attr import (
    GradientShap,
    DeepLift,
    DeepLiftShap,
    IntegratedGradients,
    LayerConductance,
    NeuronConductance,
    NoiseTunnel,
    
)
from pprint import pprint

## Useful functions

In [3]:

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print( torch.cuda.get_device_name(device) if device == 'cuda:0' else 'CPU :(' )

# create a new SM service!
info_getter = InfoGetter()

def get_source(url):
    for schema in ['', 'https://', 'http://']:
        try:
            u = schema + url
            response = requests.get(u, verify=False, timeout=5, headers={
                'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36 RuxitSynthetic/1.0 v5553715026974570137 t6281935149377429786'
            })
            return response.text, u
        except Exception as e:
            pass
    return None



CPU :(


In [4]:
# TODO for users: load your URL/labels into bot_data 
bot_data = [
    ['www.anntaylor.com', 0],
    ['bing.com', 0],
    ['google.com', 1],
    ['https://incomation.kartra.com', 1],
]

In [5]:
# create the bot data features based on ONLY saved pages!
X = []
Y = []
collected_urls = []
whois_data = {}
countries = list(json.load(open('./data/country.json', 'r', encoding='utf-8')).keys())

for ind_data, (url, label) in enumerate(bot_data):
    print('%d/%d' %(ind_data + 1, len(bot_data)), end='\r', flush=True)
        
    if 'http' in url:
        u = urlparse(url).netloc
    else:
        u = url

    # get whois data
    try:
        if url in whois_data:
            icann_data = whois_data[url]
        elif u in whois_data:
            icann_data = whois_data[u]
        else:
            icann_data = whois.whois(url)
            whois_data[url] = icann_data

        if icann_data == {} or 'creation_date' not in icann_data or icann_data['creation_date'] == None:
            continue
    except Exception as e:
        print(e)
        continue

    # get page source (from cache or DL)
    if os.path.exists(os.path.join('dataset', 'saved_pages', url.replace('/', '').replace('?', '').replace('!', '').replace('@', '').replace(':', '')) + '.html'):
        fin = open(os.path.join('dataset', 'saved_pages', url.replace('/', '').replace('?', '').replace('!', '').replace('@', '').replace(':', '') + '.html'), 'r', encoding='utf-8')
        page_content = fin.read()
        fin.close()
    elif os.path.exists(os.path.join('dataset', 'saved_pages', 'http' + url.replace('/', '').replace('?', '').replace('!', '').replace('@', '').replace(':', '')) + '.html'):
        fin = open(os.path.join('dataset', 'saved_pages', 'http' + url.replace('/', '').replace('?', '').replace('!', '').replace('@', '').replace(':', '') + '.html'), 'r', encoding='utf-8')
        page_content = fin.read()
        fin.close()
    elif os.path.exists(os.path.join('dataset', 'saved_pages', 'https' + url.replace('/', '').replace('?', '').replace('!', '').replace('@', '').replace(':', '')) + '.html'):
        fin = open(os.path.join('dataset', 'saved_pages', 'https' + url.replace('/', '').replace('?', '').replace('!', '').replace('@', '').replace(':', '') + '.html'), 'r', encoding='utf-8')
        page_content = fin.read()
        fin.close()
    elif os.path.exists(os.path.join('dataset', 'saved_pages', u.replace('/', '').replace('?', '').replace('!', '').replace('@', '').replace(':', '')) + '.html'):
        fin = open(os.path.join('dataset', 'saved_pages', u.replace('/', '').replace('?', '').replace('!', '').replace('@', '').replace(':', '') + '.html'), 'r', encoding='utf-8')
        page_content = fin.read()
        fin.close()
    elif os.path.exists(os.path.join('dataset', 'saved_pages', 'http' + u.replace('/', '').replace('?', '').replace('!', '').replace('@', '').replace(':', '')) + '.html'):
        fin = open(os.path.join('dataset', 'saved_pages', 'http' + u.replace('/', '').replace('?', '').replace('!', '').replace('@', '').replace(':', '') + '.html'), 'r', encoding='utf-8')
        page_content = fin.read()
        fin.close()
    elif os.path.exists(os.path.join('dataset', 'saved_pages', 'https' + u.replace('/', '').replace('?', '').replace('!', '').replace('@', '').replace(':', '')) + '.html'):
        fin = open(os.path.join('dataset', 'saved_pages', 'https' + u.replace('/', '').replace('?', '').replace('!', '').replace('@', '').replace(':', '') + '.html'), 'r', encoding='utf-8')
        page_content = fin.read()
        fin.close()
    else:
        try:
            page_content, url = get_source(url)
            with open('./data/saved_pages/' + url.replace('/', '').replace('?', '').replace('!', '').replace('@', '').replace(':', '') + '.html', 'w', encoding='utf-8') as fin:
                fin.write(page_content)
                fin.close()
        except Exception as e:
            print(e)
            pass

    # create features
    if page_content != None:
        X.append(create_features_single({url: icann_data}, countries, page_content, url, info_getter, './data/IP2LOCATION-LITE-DB1.BIN'))
        Y.append(label)
        collected_urls.append(u)

/opt/miniconda3/envs/pytorch/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.anntaylor.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


/opt/miniconda3/envs/pytorch/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bing.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/miniconda3/envs/pytorch/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.bing.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Error trying to connect to socket: closing socket


/opt/miniconda3/envs/pytorch/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/miniconda3/envs/pytorch/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


/opt/miniconda3/envs/pytorch/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'incomation.kartra.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/miniconda3/envs/pytorch/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'app.kartra.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/miniconda3/envs/pytorch/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'app.kartra.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [6]:
# save collected data
pickle.dump(whois_data, open('./cache/whois-cache.pkl', 'wb'))
pickle.dump((X, Y, collected_urls), open('./cache/dataset.pkl', 'wb'))

# Convert JSONL format data to features

In [5]:
def convert_to_feature(icann_data, countries, html_string, url, data_file="IP2LOCATION-LITE-DB1.BIN"):
    ip_location = IP2Location.IP2Location(data_file)
    u = urlsplit(url)
    if 'http' not in url:
        u = u.path
    else:
        u = u.netloc
    X = []

    # range
    try:
        created_date = datetime.datetime.strptime(icann_data[url]['creation_date'].split(' ')[0], '%Y-%m-%d')
        exp_date = datetime.datetime.strptime(icann_data[url]['expiration_date'].split(' ')[0], '%Y-%m-%d')
        total_age = (exp_date - created_date).days / 365.0
    except:
        total_age = -1

    # set country
    country_feature = [0 for i in range(255)]
    if 'country' in icann_data[url] and icann_data[url]['country'] in countries:
        country = countries.index(
            icann_data[url]['country']
        )
        country_feature[country] = 1
    else:
        country = -1

    # whois guard is used or not
    names = icann_data[url]['registrar'] if 'registrar' in icann_data[url] else None
    if type(names) is list:
        names = names[0]

    whois_guard_keywords = ['WhoisGuard'.lower(), 'REDACTED FOR PRIVACY'.lower(), 'Private Whois'.lower(), 'DOMAIN PRIVACY'.lower()]

    for kw in whois_guard_keywords:
        if names != None and (kw in names.lower() or 'priva' in names.lower()):
            guard = 1
            break
        elif names == None:
            guard = -1
        else:
            guard = 0

    # get features from a single page
    # parse DOM tree
    parsed_tree = BeautifulSoup(html_string, 'html.parser')
    script_tags = parsed_tree.find_all('script', src=True)
    
    link_tags = parsed_tree.find_all('a', href=True)
    num_external_links = 0

    if link_tags != None:
        for link_tag in link_tags:
            link = link_tag['href'] # if 'href' in link_tag else ''
            # check if it's external link
            if link.find('http') == 0 and u not in link:
                num_external_links += 1
    
    # Shopping sites only filter: filter sites with payment
    # if 'payment' not in html_string.lower() and 'cart' not in html_string.lower():
    #     continue
    
    # check social media links
    total_social_medias = [-1, -1, -1]
    social_media_regex = [
        r'instagram\.com\/[a-zA-Z0-9_\-]+',
        r'facebook\.com\/[a-zA-Z0-9_\-]+',
        r'twitter\.com\/[a-zA-Z0-9_\-]+',
    ]
    
    for i, r in enumerate(social_media_regex):
        found = re.findall(r, html_string)
        if len(found) > 0: # and 'login' not in found[0]:
            if found[0].split('/')[-1] in url:
                total_social_medias[i] = 1
            else:
                total_social_medias[i] = 0

    # Host country
    host_country_feature = [0 for i in range(255)]
    try:
        host_ip = socket.gethostbyname(u)
        response = ip_location.get_all(host_ip)
        host_country = response.country_short.decode()
        host_domain_same = 1 if countries.index(host_country) == country else 0
        host_country = countries.index(host_country)
        host_country_feature[host_country] = 1
    except:
        host_domain_same = -1
        host_country = -1

    has_digit = False
    for i in url:
        if i.isdigit():
            has_digit = True
            break

    is_cheap = 0
    cheap_registrars = ['Namecheap', 'GoDaddy', 'Porkbun', 'NameSilo', 'Danesco', 'Hostinger']
    
    for cr in cheap_registrars:
        if 'registrar' in i and i['registrar'].lower() is not None and cr.lower() in i['registrar'].lower():
            is_cheap = 1
            break
    
    top_cheap_domains = ['club', 'buzz', 'xyz', 'ua', 'icu', 'space', 'agency', 'monster', 'pw', 'click', 'website', 'site', 'club', 'online', 'link', 'shop', 'feedback', 'uno', 'press', 'best', 'fun', 'host', 'store', 'tech', 'top', 'it']
    uses_cheap_domain = 0
    for tld in top_cheap_domains:
        if tld in url:
            uses_cheap_domain = 1
            
    # domain in text
    if parsed_tree.find('body') is not None:
        domain_in_text = parsed_tree.find('body').text.count(u)
    else:
        domain_in_text = -1
        
    # append feature vector to dataset
    domain_name = '.'.join(u.split('.')[:-1])
    
    X.append([
        guard, 
        total_social_medias[0], 
        total_social_medias[1], 
        total_social_medias[2], 
        num_external_links, 
        host_domain_same,
        len(script_tags),
        1 if '-' in url else 0,
        domain_name.count('.'),
        1 if has_digit else 0,
        is_cheap,
        uses_cheap_domain,
        domain_in_text,
        0,
        1 if u.split('.')[-1] not in ['com', 'net', 'org', 'uk', 'gov', 'au'] else 0,
        total_age,
    ])

    X[-1].extend(country_feature)
    X[-1].extend(host_country_feature)

    # count missing features
    missings = 0

    for i in X[-1]:
        if i == -1:
            missings += 1
    if country == -1:
        missings += 1
    if host_country == -1:
        missings += 1

    X[-1].append(missings)

    return X


In [7]:
whois_data = {}
bot_data = []
processed_dataset = []

countries = list(json.load(open('./data/country.json', 'r', encoding='utf-8')).keys())
countries_pd = pd.read_csv('./data/country.csv')
print(countries[:10])
countries_pd.head()
countries_dict = {}
for row in countries_pd.iterrows():
    countries_dict[row[1]['value'].lower()] = row[1]['id']

with open('./dataset.jsonl', 'r', encoding='utf-8') as fin:
    for line in fin.readlines():
        data = json.loads(line)
        bot_data.append(data)

for ind_data, info in tqdm(enumerate(bot_data), total=len(bot_data)):
    sample_features = convert_to_feature(
        {info['url']: info['whois']},
        countries,
        info['content'],
        info['url'],
        data_file='./data/IP2LOCATION-LITE-DB1.BIN'
    )
    
    processed_dataset.append((
        sample_features, info['label'], info['url']
    ))

# save collected data
pickle.dump(processed_dataset, open('./cache/dataset.pkl', 'wb'))

['AF', 'AX', 'AL', 'DZ', 'AS', 'AD', 'AO', 'AI', 'AQ', 'AG']


  1%|          | 116/10087 [00:28<20:46,  8.00it/s] 